In [1]:
import sys
sys.path.append('../src')
from data import get_documents_list, get_cleaned_dataframe_with_topics, get_cleaned_dataframe, get_sentences_and_labels_lists
from models import classification, get_embedding_model, get_BERTopic_model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


loaded spacy language model: de_core_news_sm


For monolingual execution, please change WIKI_LANGUAGES in config.ini file to language code of language you want to use

load the preprocessed texts in defined languages into a dataframe

In [2]:
#df = get_cleaned_dataframe_with_topics() #todo:fix
df = get_cleaned_dataframe()
documents = get_documents_list('cleaned_sentences')

Decide what types of documents you want to train your embedding model on

## Load Embedding Model
Choose method and text_type the document was trained on.
### Parameters:
- method = "muse", "fastText", or "Word2Vec"
- Optional: text_type = "cleaned_texts", "cleaned_texts_topics", ... (see more in embedding_models.py)
- Optional: training_type = "pretrained_aligned" or "from_scratch"

e.g. use method = "muse", text_type='cleaned_texts', training_type = "pretrained_aligned" for pretrained aligned models downloaded from facebookresearch MUSE website

In [3]:
model, word_vectors = get_embedding_model(documents, text_type = "cleaned_sentences", method = "fastText")

Model:  /home/Bachelor-Thesis/models/classification/FastText/FastText_de_cleaned_sentences  loaded


In [4]:
word_vectors.similarity('composer', 'komponist')

0.9999891

## Split data into train and test sets


In [5]:
from sklearn.model_selection import train_test_split
# case for full texts:
#X_train, X_test, y_train, y_test = train_test_split(df['cleaned_texts'], df['label'] , test_size=0.2)

# case for sentences
docs, targets = get_sentences_and_labels_lists()
X_train, X_test, y_train, y_test = train_test_split(docs, targets , test_size=0.2)

import pandas as pd
X_train = pd.Series(X_train) 
X_test = pd.Series(X_test) 
y_train = pd.Series(y_train) 
y_test = pd.Series(y_test) 

In [ ]:
X_train_vect = classification.text2vec(X_train, word_vectors)
X_test_vect = classification.text2vec(X_test, word_vectors)
X_train_vect_avg = classification.average_text_vector(X_train_vect)
X_test_vect_avg = classification.average_text_vector(X_test_vect)

## Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy="not majority")
X_train_2d = X_train.values.reshape(-1, 1) # puts each elem of array in own array
X_res, y_res = ros.fit_resample(X_train_2d, y_train)

In [ ]:
print(y_res.value_counts())
y_res.value_counts().plot.pie(autopct='%0.2f')

In [ ]:
#X_res revert 2_dimensionality
X_res_1d = X_res.flatten()

# classification with resampled vectors
X_res_vect = classification.text2vec(X_res_1d, word_vectors)
X_res_vect_avg = classification.average_vector(X_res_vect)

In [ ]:
y_res_pred = classification.classify_predict(X_res_vect_avg, X_test_vect_avg, y_res, "Random Forest")
classification.print_classification_report(y_test, y_res_pred)
y_res_pred = classification.classify_predict(X_res_vect_avg, X_test_vect_avg, y_res, "Multinomial Naive Bayes")
classification.print_classification_report(y_test, y_res_pred)
y_res_pred = classification.classify_predict(X_res_vect_avg, X_test_vect_avg, y_res, "SVM")
classification.print_classification_report(y_test, y_res_pred)